# Visualizing results

Here we explore various options for visualizing the results of our analysis and model fitting.

Primarily we will be using Plotly, which is a powerful interactive plotting library. We will also use Matplotlib for some simple plots.

Also we will see about using pyLDAvis, which is a library for visualizing the results of topic modeling.

## Scattertext library

Scattertext is a library for visualizing text data. It is particularly useful for visualizing the differences between two corpora. Here we will use it to visualize the differences between the trial documents in our corpora.

In [ ]:
# We will be using tutorial here at
# https://github.com/JasonKessler/scattertext#visualizing-phrase-associations-with-scattertext-and-spacy

# first we will need to isntall scattertext


In [2]:
# we will need pandas and spacy as well
import pandas as pd
import spacy
import scattertext as st

df = st.SampleCorpora.ConventionData2012.get_data().assign(
    parse=lambda df: df.text.apply(st.whitespace_nlp_with_sentences)
)

corpus = st.CorpusFromParsedDocuments(
    df, category_col='party', parsed_col='parse'
).build().get_unigram_corpus().compact(st.AssociationCompactor(2000))

html = st.produce_scattertext_explorer(
    corpus,
    category='democrat', category_name='Democratic', not_category_name='Republican',
    minimum_term_frequency=0, pmi_threshold_coefficient=0,
    width_in_pixels=1000, metadata=corpus.get_df()['speaker'],
    transform=st.Scalers.dense_rank
)
open('./demo_compact.html', 'w', encoding="utf-8").write(html)


1664109